# Astropy and SunPy - A Quick Primer

Before we progress to searching for DKIST data we need to cover some functionality of the [SunPy](https://sunpy.org) and [Astropy](https://astropy.org) packages.
These two packages, amongst a few others, are the core components of the DKIST tools.
In this session we shall only quickly cover the functionality of SunPy and Astropy we need for the rest of the workshop.
There are many other parts of these packages which are useful when working with DKIST data, which you should explore.

As we covered in the introduction, Python is a modular language; Astropy provides fundamental functionality for a lot of different types of astronomy.
In this section we shall cover the following parts of Astropy:

* Units
* World Coordinate Systems
* Coordinates

## Units

Astropy provides a subpackage {obj}`astropy.units` which provides tools for associating physical units with numbers and arrays.
This lets you do mathematical operations on these arrays while propagating the units.

To get started we import `astropy.units`; because we are going to be using this a lot, we import it as `u`.

In [1]:
import astropy.units as u

Now we can access various units as such:

In [3]:
u.meter

Unit("m")

In [4]:
u.kg

Unit("kg")

We can now attach a unit to a number:

In [6]:
length = 10 * u.m
length

<Quantity 10. m>

We can also compose multiple quantities (a number with a unit):

In [7]:
speed = length / (30 * u.min)
speed

<Quantity 0.33333333 m / min>

Using the `.to()` method on a `u.Quantity` object lets you convert a quantity to a different unit.

In [8]:
speed.to(u.km/u.h)

<Quantity 0.02 km / h>

In [9]:
speed.to(u.m/u.s)

<Quantity 0.00555556 m / s>

### Equivalencies

Some conversions are not done by a conversion factor as between miles and kilometers – for example converting between wavelength and frequency:

In [10]:
(656.281 * u.nm).to(u.Hz)

UnitConversionError: 'nm' (length) and 'Hz' (frequency) are not convertible

However we can make use of a spectral *equivalency* to indicate the link between the units:

In [12]:
(656.281 * u.nm).to(u.Hz, equivalencies=u.spectral())

<Quantity 4.56805024e+14 Hz>

### Constants

The `astropy.constants` sub-package provides a set of physical constants which are compatible with the units/quantities framework:

In [13]:
from astropy.constants import M_sun, c

In [14]:
E = M_sun * c ** 2
E

<Quantity 1.78709367e+47 m2 kg / s2>

In [15]:
E.to(u.J)

<Quantity 1.78709367e+47 J>

In [16]:
E.decompose()

<Quantity 1.78709367e+47 m2 kg / s2>

## Coordinates

The Astropy coordinates submodule {obj}`astropy.coordinates` provides classes to represent physical coordinates with all their associated metadata, and transform them between different coordinate systems.
Currently, {obj}`astropy.coordinates` supports: 

* Spatial coordinates via {obj}`astropy.coordinates.SkyCoord`
* Spectral coordinates via {obj}`astropy.coordinates.SpectralCoord`
* Stokes profiles via {obj}`astropy.coordinates.StokesCoord` (coming soon)

### Spatial Coordinates

SunPy provides extensions to the Astropy coordinate system to represent common solar physics frames.
So to use the sunpy coordinates we have to first import {obj}`sunpy.coordinates` which registers them with astropy.

In [17]:
import sunpy.coordinates
from astropy.coordinates import SkyCoord

We can now create a `SkyCoord` object representing a point on the Sun:

In [18]:
SkyCoord(10*u.arcsec, 20*u.arcsec, frame="helioprojective")

<SkyCoord (Helioprojective: obstime=None, rsun=695700.0 km, observer=None): (Tx, Ty) in arcsec
    (10., 20.)>

In [21]:
SkyCoord(10*u.m, 10*u.m, 10*u.m, frame="heliocentric")

<SkyCoord (Heliocentric: obstime=None, observer=None): (x, y, z) in m
    (10., 10., 10.)>

This is the most minimal version of this coordinate frame, however, it isn't very useful as we haven't provided enough information to be able to transform it to other frames.
This is because helioprojective is an observer centred coordinate frame, so we need to know where in inertial space the observer is.
One way of doing this is to say the observer is on Earth at a specific time:

In [24]:
hpc1 = SkyCoord(10*u.arcsec, 20*u.arcsec, frame="helioprojective", obstime="2023-05-21T04:00:00", observer="earth")
hpc1

<SkyCoord (Helioprojective: obstime=2023-05-21T04:00:00.000, rsun=695700.0 km, observer=<HeliographicStonyhurst Coordinate for 'earth'>): (Tx, Ty) in arcsec
    (10., 20.)>

This coordinate can now be converted to other frames, such as heliographic coordinates:

In [29]:
hgs = hpc1.transform_to("heliographic_stonyhurst")
hgs

<SkyCoord (HeliographicStonyhurst: obstime=2023-05-21T04:00:00.000, rsun=695700.0 km): (lon, lat, radius) in (deg, deg, AU)
    (0.60176385, -0.81326383, 0.00465047)>

In [31]:
hgs.lon

<Longitude 0.60176385 deg>

In [32]:
hgs.lat

<Latitude -0.81326383 deg>

In [33]:
hgs.radius

<Distance 0.00465047 AU>

In [27]:
hpc1.Tx

<Longitude 10. arcsec>

In [28]:
hpc1.Ty

<Latitude 20. arcsec>

There are few things to notice about the difference between these two `SkyCoord` objects:

1. The default representation of the latitude and longitude is now in degrees as is conventional.
1. The heliographic frame is three dimensional (it has a radius), when the input frame was not. This is because the distance from the observer was calculated using the `rsun` attribute.
1. The `obstime` and `rsun` attributes are still present, but the `observer` attribute isn't. This is because heliographic coordinates are not observer dependent.
1. The `obstime` attribute is still important to transform to other frames, as the heliographic frame needs to know the location of Earth.

### Spectral Coordinates

{obj}`astropy.coordinates.SpectralCoord` is a `Quantity`-like object which also holds information about the observer and target coordinates and relative velocities.

```{note}
Use of `SpectralCoord` with solar data is still experimental so not all features may work, or be accurate.
```

In [34]:
from astropy.coordinates import SpectralCoord
from sunpy.coordinates import get_earth

`SpectralCoord` does not automatically make the HPC coordinate 3D, but wants the distance, so we do it explicitally:

In [36]:
hpc2 = hpc1.make_3d()
hpc2

<Helioprojective Coordinate (obstime=2023-05-21T04:00:00.000, rsun=695700.0 km, observer=<HeliographicStonyhurst Coordinate for 'earth'>): (Tx, Ty, distance) in (arcsec, arcsec, AU)
    (10., 20., 1.00733392)>

Now we can construct our spectral coordinate with both a target and an observer

In [37]:
spc = SpectralCoord(586.3 * u.nm, target=hpc2, observer=get_earth(time=hpc2.obstime))
spc

<SpectralCoord 
   (observer: <HeliographicStonyhurst Coordinate (obstime=2023-05-21T04:00:00.000, rsun=695700.0 km): (lon, lat, radius) in (deg, deg, AU)
                  (0., -2.0168479, 1.0119831)
               (d_lon, d_lat, d_radius) in (arcsec / s, arcsec / s, km / s)
                  (0., 0., 0.)>
    target: <Helioprojective Coordinate (obstime=2023-05-21T04:00:00.000, rsun=695700.0 km, observer=<HeliographicStonyhurst Coordinate for 'earth'>): (Tx, Ty, distance) in (arcsec, arcsec, AU)
                (10., 20., 1.00733392)
             (d_Tx, d_Ty, d_distance) in (arcsec / s, arcsec / s, km / s)
                (0., 0., 0.)>
    observer to target (computed from above):
      radial_velocity=0.00898274411761113 km / s
      redshift=2.996320969117505e-08)
  586.3 nm>

We can show the full details of the spectral coord (working around a [bug in astropy](https://github.com/astropy/astropy/issues/14758)):

In [38]:
spc.to(u.Hz)

<SpectralCoord 
   (observer: <HeliographicStonyhurst Coordinate (obstime=2023-05-21T04:00:00.000, rsun=695700.0 km): (lon, lat, radius) in (deg, deg, AU)
                  (0., -2.0168479, 1.0119831)
               (d_lon, d_lat, d_radius) in (arcsec / s, arcsec / s, km / s)
                  (0., 0., 0.)>
    target: <Helioprojective Coordinate (obstime=2023-05-21T04:00:00.000, rsun=695700.0 km, observer=<HeliographicStonyhurst Coordinate for 'earth'>): (Tx, Ty, distance) in (arcsec, arcsec, AU)
                (10., 20., 1.00733392)
             (d_Tx, d_Ty, d_distance) in (arcsec / s, arcsec / s, km / s)
                (0., 0., 0.)>
    observer to target (computed from above):
      radial_velocity=0.00898274411761113 km / s
      redshift=2.996320969117505e-08)
  5.11329452e+14 Hz>

In [39]:
print(repr(spc))

<SpectralCoord 
   (observer: <HeliographicStonyhurst Coordinate (obstime=2023-05-21T04:00:00.000, rsun=695700.0 km): (lon, lat, radius) in (deg, deg, AU)
                  (0., -2.0168479, 1.0119831)
               (d_lon, d_lat, d_radius) in (arcsec / s, arcsec / s, km / s)
                  (0., 0., 0.)>
    target: <Helioprojective Coordinate (obstime=2023-05-21T04:00:00.000, rsun=695700.0 km, observer=<HeliographicStonyhurst Coordinate for 'earth'>): (Tx, Ty, distance) in (arcsec, arcsec, AU)
                (10., 20., 1.00733392)
             (d_Tx, d_Ty, d_distance) in (arcsec / s, arcsec / s, km / s)
                (0., 0., 0.)>
    observer to target (computed from above):
      radial_velocity=0.00898274411761113 km / s
      redshift=2.996320969117505e-08)
  586.3 nm>


## World Coordinate System

One of the other core components of the ecosystem provided by Astropy is the {obj}`astropy.wcs` package which provides tools for mapping pixel to world coordinates and world to pixel.
When loading a FITS file with complete (and standard-compliant) WCS metadata we can create an `astropy.wcs.WCS` object.
For this example we will use a single VISP frame.

In [40]:
import sunpy.coordinates

To read this FITS file we will use {obj}`astropy.io.fits`.

In [41]:
from astropy.io import fits

hdu_list = fits.open("./data/VISP_2022_10_24T19_47_33_218_00630205_I_BEOGN_L1.fits")

In [42]:
hdu_list.info()

Filename: ./data/VISP_2022_10_24T19_47_33_218_00630205_I_BEOGN_L1.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       6   ()      
  1  observation    1 CompImageHDU    305   (2555, 976, 1)   float64   


We can now access the header of the second HDU (the one containing the data):

In [43]:
hdu_list[1].header

XTENSION= 'IMAGE   '                                                            
BITPIX  =                  -64                                                  
NAXIS   =                    3                                                  
NAXIS1  =                 2555 / [pix]                                          
NAXIS2  =                  976 / [pix]                                          
NAXIS3  =                    1 / [pix]                                          
PCOUNT  =                    0                                                  
GCOUNT  =                    1                                                  
BUNIT   = 'ct      '                                                            
DATE    = '2023-04-21T20:11:35.439'                                             
DATE-BEG= '2022-10-24T19:47:33.218'                                             
DATE-END= '2022-10-24T19:47:33.484'                                             
TELAPSE =   0.26600011624395

In [46]:
u.Unit(hdu_list[1].header["CUNIT2"])

Unit("nm")

Using this header we can create a `astropy.wcs.WCS` object:

In [44]:
from astropy.wcs import WCS

wcs = WCS(hdu_list[1].header)
wcs

Set MJD-BEG to 59876.824690 from DATE-BEG.
Set MJD-AVG to 59876.824692 from DATE-AVG.
Set MJD-END to 59876.824693 from DATE-END'. [astropy.wcs.wcs]
Set OBSGEO-B to    20.706700 from OBSGEO-[XYZ].
Set OBSGEO-H to     3063.997 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


WCS Keywords

Number of WCS axes: 3
CTYPE : 'HPLT-TAN'  'AWAV'  'HPLN-TAN'  
CRVAL : 0.05511659716785147  6.302051000000001e-07  -0.11045442200016803  
CRPIX : 1177.642614956193  -23.0  538.5172545826539  
PC1_1 PC1_2 PC1_3  : 0.9896333838201206  0.0  0.008168793880372888  
PC2_1 PC2_2 PC2_3  : 0.0  1.0  0.0  
PC3_1 PC3_2 PC3_3  : -0.00247072062532277  0.0  0.9898254254995873  
CDELT : 8.277777777777778e-06  1.62511509639976e-12  1.48372329380796e-05  
NAXIS : 2555  976  1

This WCS object allows us to convert between pixel and world coordinates, for example:

In [45]:
wcs.pixel_to_world(0, 0, 0)

[<SkyCoord (Helioprojective: obstime=2022-10-24T19:47:33.351, rsun=695700.0 km, observer=<HeliographicStonyhurst Coordinate (obstime=2022-10-24T19:47:33.351, rsun=695700.0 km): (lon, lat, radius) in (deg, deg, m)
     (0.00173993, 5.09237547, 1.48791805e+11)>): (Tx, Ty) in arcsec
     (-425.89949011, 163.58844533)>,
 <SpectralCoord 
    (observer: <ITRS Coordinate (obstime=59876.8246915625, location=(0., 0., 0.) km): (x, y, z) in m
                   (-5466045.25695494, -2404388.73741278, 2242133.88769004)
                (v_x, v_y, v_z) in km / s
                   (0., 0., 0.)>
     target: <Helioprojective Coordinate (obstime=2022-10-24T19:47:33.351, rsun=695700.0 km, observer=<HeliographicStonyhurst Coordinate (obstime=2022-10-24T19:47:33.351, rsun=695700.0 km): (lon, lat, radius) in (deg, deg, m)
                 (0.00173993, 5.09237547, 1.48791805e+11)>): (Tx, Ty, distance) in (arcsec, arcsec, kpc)
                 (-397.6359192, 198.4197498, 1000.)
              (d_Tx, d_Ty, d_d

This call returns a {obj}`astropy.coordinates.SkyCoord` object (which needs sunpy to be imported), we will come onto this more later.

We can also convert between pixel and plain numbers:

In [47]:
wcs.pixel_to_world_values(0, 0, 0)

(array(0.04544123), array(6.30244103e-07), array(-0.11830541))

The units for these values are given by:

In [49]:
wcs.world_axis_units

['deg', 'm', 'deg']

The WCS also has information about what the world axes represent:

In [50]:
wcs.world_axis_physical_types

['custom:pos.helioprojective.lat', 'em.wl', 'custom:pos.helioprojective.lon']

We can also inspect the correlation between the world axes and pixel axes:

In [51]:
wcs.axis_correlation_matrix

array([[ True, False,  True],
       [False,  True, False],
       [ True, False,  True]])

This correlation matrix has the world dimensions as rows, and the pixel dimensions as columns.
Here we have a 2D image, with two pixel and two world axes where both are coupled together.
This means that to calculate either latitude or longitude you need both pixel coordinates.